# Trabalho Final

**Aluno**: Lucas de Miranda Rodrigues

**Matrícula**: 96143



## 05 - Tratamento da base *experiencia_trabalho.csv*

Essa base tem os 111 prefis totais e descreve o histórico de experiência profissional desses perfis. A coluna **company** infoma o nome da empresa que o perfil trabalhou e/ou trabalha. E as colunas **date**, **location** e **posicion** informam, respectivamente, o período de adimissão e demissão do profission, a localização e o cargo acupado.

Tem 497 registros, então um perfil pode ter mais de uma experiência profissional. E a base contem valores ausentes.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

experiencia = pd.read_csv("bases/experiencia_trabalho.csv")

# Análise inicial
print("Nº de registros:", len(experiencia))
print("Nº de valores únicos de perfil_id:", experiencia.perfil_id.nunique())
print("Nº de valores únicos de company:", experiencia.company.nunique())
print("Nº de valores únicos de location:", experiencia.location.nunique())
print("Nº de valores únicos de position:", experiencia.position.nunique())
print("Nº de valores vazios de company:", experiencia.company.isnull().sum())
print("Nº de valores vazios de date:", experiencia.date.isnull().sum())
print("Nº de valores vazios de location:", experiencia.location.isnull().sum())
print("Nº de valores vazios de position:", experiencia.position.isnull().sum())

experiencia.describe(include="all")

Nº de registros: 497
Nº de valores únicos de perfil_id: 111
Nº de valores únicos de company: 2
Nº de valores únicos de location: 71
Nº de valores únicos de position: 386
Nº de valores vazios de company: 25
Nº de valores vazios de date: 26
Nº de valores vazios de location: 160
Nº de valores vazios de position: 25


,company,date,location,position,perfil_id
count,472,471,337,472,497.000000
unique,2,328,71,386,NaN
top,Any Company,mar de 2017 – o momento,"Belo Horizonte e Região, Brasil",Software Developer,NaN
freq,318,21,138,9,NaN
mean,NaN,NaN,NaN,NaN,54.187123
std,NaN,NaN,NaN,NaN,32.199717
min,NaN,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,NaN,27.000000
50%,NaN,NaN,NaN,NaN,54.000000
75%,NaN,NaN,NaN,NaN,81.000000


In [2]:
experiencia = experiencia[(experiencia.company.notnull()) & (experiencia.position.notnull()) & (experiencia.date.notnull())]
print("Nº de valores únicos de perfil_id:", experiencia.perfil_id.nunique())
print("Nº de registros restantes após remoção dos vazios:", len(experiencia))
print("Nº de valores vazios de company:", experiencia.company.isnull().sum())
print("Nº de valores vazios de date:", experiencia.date.isnull().sum())
print("Nº de valores vazios de location:", experiencia.location.isnull().sum())
print("Nº de valores vazios de position:", experiencia.position.isnull().sum())

Nº de valores únicos de perfil_id: 110
Nº de registros restantes após remoção dos vazios: 471
Nº de valores vazios de company: 0
Nº de valores vazios de date: 0
Nº de valores vazios de location: 134
Nº de valores vazios de position: 0


In [3]:
experiencia.head(10)

,company,date,location,position,perfil_id
0,Trojan Technologies,set de 2011 – o momento,"Belo Horizonte e Região, Brasil",CEO and Founder,0
6,Any Company,mar de 2016 – o momento,NaN,Endeavor Entrepreneur,0
7,Any Company,out de 2015 – o momento,"Belo Horizonte e Região, Brasil",Investor and Board Member,0
8,Any Company,out de 2012 – mai de 2013,"Santiago, Chile",Start-Up Chile Entrepreneur,0
9,Any Company,set de 2009 – jan de 2011,"São Paulo e Região, Brasil",Founder and Sales Director,0
10,Trojan Technologies,jul de 2017 – o momento,"Belo Horizonte e Região, Brasil",Product Manager,1
11,Any Company,jan de 2017 – o momento,"Belo Horizonte, Minas Gerais",Diretora de Whatever,1
12,Trojan Technologies,mar de 2017 – jul de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de Onboarding,1
13,Any Company,dez de 2016 – mar de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de aceleração,1
14,Any Company,jun de 2016 – jan de 2017,"Belo Horizonte e Região, Brasil",Subcoordenadora de Desenvolvimento de Negócios...,1


## 05.1 - Tratamento das data

A coluna **date** pode ser quebrada no ano de início (**inicio**) e no ano de fim (**fim**) da experiência. As experiências em aberto, possuem o texto "o momento" que será interpretado como o mês/ano da extração dos dados (09/2017). As experiências em aberto serão marcadas na coluna booleana **atual**, que indicando que o funcionário ainda trabalha na trojan.

In [4]:
#Cria a coluna: data_controle por causa da função split
experiencia['data_controle']=experiencia.date.str.split(' ')

def separacao_inicio(x):
    """Função que separa a data inicial e dá a resposta
    através de um vetor"""
    v_m=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    v_n=['01','02','03','04','05','06','07','08','09','10','11','12']
    v_valor=np.arange(12)
    a='01/'+x[0]
    for i in v_valor:
        if x[0]==v_m[i]:
            a=v_n[i] + '/' + x[2]
    return datetime.strptime(a,'%m/%Y')

#Cria a coluna: inicio       
experiencia['inicio']=experiencia['data_controle'].map(separacao_inicio)

def separacao_fim(x):
    """Função que separa a data final e dá a resposta
    através de um vetor"""
    v_m=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    v_n=['01','02','03','04','05','06','07','08','09','10','11','12']
    v_valor=np.arange(12)
    a='01/'+x[2]
    if x[len(x)-1]=='momento':
        a='09/2017'
    elif len(x)==7:
        for i in v_valor:
            if x[4]==v_m[i]: 
                a=v_n[i] + '/' + x[len(x)-1]
    elif len(x)==5:
        a='01/'+x[4]
        for i in v_valor:
            if x[2]==v_m[i]: 
                a=v_n[i] + '/' + x[len(x)-1]
    return datetime.strptime(a,'%m/%Y')

#Cria a coluna: fim
experiencia['fim']=experiencia['data_controle'].map(separacao_fim)

#Mostra o resultado
experiencia.head(10)

,company,date,location,position,perfil_id,data_controle,inicio,fim
0,Trojan Technologies,set de 2011 – o momento,"Belo Horizonte e Região, Brasil",CEO and Founder,0,"[set, de, 2011, –, o, momento]",2011-09-01,2017-09-01
6,Any Company,mar de 2016 – o momento,NaN,Endeavor Entrepreneur,0,"[mar, de, 2016, –, o, momento]",2016-03-01,2017-09-01
7,Any Company,out de 2015 – o momento,"Belo Horizonte e Região, Brasil",Investor and Board Member,0,"[out, de, 2015, –, o, momento]",2015-10-01,2017-09-01
8,Any Company,out de 2012 – mai de 2013,"Santiago, Chile",Start-Up Chile Entrepreneur,0,"[out, de, 2012, –, mai, de, 2013]",2012-10-01,2013-05-01
9,Any Company,set de 2009 – jan de 2011,"São Paulo e Região, Brasil",Founder and Sales Director,0,"[set, de, 2009, –, jan, de, 2011]",2009-09-01,2011-01-01
10,Trojan Technologies,jul de 2017 – o momento,"Belo Horizonte e Região, Brasil",Product Manager,1,"[jul, de, 2017, –, o, momento]",2017-07-01,2017-09-01
11,Any Company,jan de 2017 – o momento,"Belo Horizonte, Minas Gerais",Diretora de Whatever,1,"[jan, de, 2017, –, o, momento]",2017-01-01,2017-09-01
12,Trojan Technologies,mar de 2017 – jul de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de Onboarding,1,"[mar, de, 2017, –, jul, de, 2017]",2017-03-01,2017-07-01
13,Any Company,dez de 2016 – mar de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de aceleração,1,"[dez, de, 2016, –, mar, de, 2017]",2016-12-01,2017-03-01
14,Any Company,jun de 2016 – jan de 2017,"Belo Horizonte e Região, Brasil",Subcoordenadora de Desenvolvimento de Negócios...,1,"[jun, de, 2016, –, jan, de, 2017]",2016-06-01,2017-01-01


In [5]:
#Cria a coluna:atual
experiencia['atual']=False
experiencia.loc[experiencia.company.str.contains('Trojan', case=False) & experiencia.date.str.contains('momento'),'atual']=True

#Mostra o resultado
experiencia.head(10)

,company,date,location,position,perfil_id,data_controle,inicio,fim,atual
0,Trojan Technologies,set de 2011 – o momento,"Belo Horizonte e Região, Brasil",CEO and Founder,0,"[set, de, 2011, –, o, momento]",2011-09-01,2017-09-01,True
6,Any Company,mar de 2016 – o momento,NaN,Endeavor Entrepreneur,0,"[mar, de, 2016, –, o, momento]",2016-03-01,2017-09-01,False
7,Any Company,out de 2015 – o momento,"Belo Horizonte e Região, Brasil",Investor and Board Member,0,"[out, de, 2015, –, o, momento]",2015-10-01,2017-09-01,False
8,Any Company,out de 2012 – mai de 2013,"Santiago, Chile",Start-Up Chile Entrepreneur,0,"[out, de, 2012, –, mai, de, 2013]",2012-10-01,2013-05-01,False
9,Any Company,set de 2009 – jan de 2011,"São Paulo e Região, Brasil",Founder and Sales Director,0,"[set, de, 2009, –, jan, de, 2011]",2009-09-01,2011-01-01,False
10,Trojan Technologies,jul de 2017 – o momento,"Belo Horizonte e Região, Brasil",Product Manager,1,"[jul, de, 2017, –, o, momento]",2017-07-01,2017-09-01,True
11,Any Company,jan de 2017 – o momento,"Belo Horizonte, Minas Gerais",Diretora de Whatever,1,"[jan, de, 2017, –, o, momento]",2017-01-01,2017-09-01,False
12,Trojan Technologies,mar de 2017 – jul de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de Onboarding,1,"[mar, de, 2017, –, jul, de, 2017]",2017-03-01,2017-07-01,False
13,Any Company,dez de 2016 – mar de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de aceleração,1,"[dez, de, 2016, –, mar, de, 2017]",2016-12-01,2017-03-01,False
14,Any Company,jun de 2016 – jan de 2017,"Belo Horizonte e Região, Brasil",Subcoordenadora de Desenvolvimento de Negócios...,1,"[jun, de, 2016, –, jan, de, 2017]",2016-06-01,2017-01-01,False


A informação de duração da experiência (**duracao**) é derivada calculando a diferença do fim ao início em meses.

In [6]:
# Calcula a duração estimada de cada experiência em meses
for_vetor=experiencia.index
r_vetor=np.zeros(len(experiencia.fim))
j=0
for i in for_vetor:
    r_vetor[j]=(experiencia.fim.loc[i].year - experiencia.inicio.loc[i].year) * 12 + experiencia.fim.loc[i].month - experiencia.inicio.loc[i].month
    j+=1

#Cria a coluna: duracao
experiencia['duracao']=r_vetor

#Mostra o resultado
experiencia.head(10)

,company,date,location,position,perfil_id,data_controle,inicio,fim,atual,duracao
0,Trojan Technologies,set de 2011 – o momento,"Belo Horizonte e Região, Brasil",CEO and Founder,0,"[set, de, 2011, –, o, momento]",2011-09-01,2017-09-01,True,72.0
6,Any Company,mar de 2016 – o momento,NaN,Endeavor Entrepreneur,0,"[mar, de, 2016, –, o, momento]",2016-03-01,2017-09-01,False,18.0
7,Any Company,out de 2015 – o momento,"Belo Horizonte e Região, Brasil",Investor and Board Member,0,"[out, de, 2015, –, o, momento]",2015-10-01,2017-09-01,False,23.0
8,Any Company,out de 2012 – mai de 2013,"Santiago, Chile",Start-Up Chile Entrepreneur,0,"[out, de, 2012, –, mai, de, 2013]",2012-10-01,2013-05-01,False,7.0
9,Any Company,set de 2009 – jan de 2011,"São Paulo e Região, Brasil",Founder and Sales Director,0,"[set, de, 2009, –, jan, de, 2011]",2009-09-01,2011-01-01,False,16.0
10,Trojan Technologies,jul de 2017 – o momento,"Belo Horizonte e Região, Brasil",Product Manager,1,"[jul, de, 2017, –, o, momento]",2017-07-01,2017-09-01,True,2.0
11,Any Company,jan de 2017 – o momento,"Belo Horizonte, Minas Gerais",Diretora de Whatever,1,"[jan, de, 2017, –, o, momento]",2017-01-01,2017-09-01,False,8.0
12,Trojan Technologies,mar de 2017 – jul de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de Onboarding,1,"[mar, de, 2017, –, jul, de, 2017]",2017-03-01,2017-07-01,False,4.0
13,Any Company,dez de 2016 – mar de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de aceleração,1,"[dez, de, 2016, –, mar, de, 2017]",2016-12-01,2017-03-01,False,3.0
14,Any Company,jun de 2016 – jan de 2017,"Belo Horizonte e Região, Brasil",Subcoordenadora de Desenvolvimento de Negócios...,1,"[jun, de, 2016, –, jan, de, 2017]",2016-06-01,2017-01-01,False,7.0


Algumas informações consolidadas por funcionários podem ser extraídas dessa base. É possível extrair o tempo de carreira na empresa em meses (**tempo_empresa_meses**), o tempo de carreira no mercado em geral em meses (**tempo_mercado_meses**), e também a quantidade de empregos que o funcionário já teve (**qtd_empregos**), o que pode auxiliar em algo na análise. Portanto, obtenha essas informações.

In [7]:
#Prepara as colunas:qtd_emprego,tempo_mercado_meses,tempo_empresa_meses e data de admissão
experiencia['qtd_empregos']=1
experiencia['tempo_mercado_meses']=experiencia.duracao
experiencia['tempo_empresa_meses_any']=0
experiencia['tempo_empresa_meses_trojan']=0
experiencia['cargo_atual']=np.nan
experiencia['data_admissao']=np.nan

#Mostra o resultado
experiencia.head(10)

,company,date,location,position,perfil_id,data_controle,inicio,fim,atual,duracao,qtd_empregos,tempo_mercado_meses,tempo_empresa_meses_any,tempo_empresa_meses_trojan,cargo_atual,data_admissao
0,Trojan Technologies,set de 2011 – o momento,"Belo Horizonte e Região, Brasil",CEO and Founder,0,"[set, de, 2011, –, o, momento]",2011-09-01,2017-09-01,True,72.0,1,72.0,0,0,NaN,NaN
6,Any Company,mar de 2016 – o momento,NaN,Endeavor Entrepreneur,0,"[mar, de, 2016, –, o, momento]",2016-03-01,2017-09-01,False,18.0,1,18.0,0,0,NaN,NaN
7,Any Company,out de 2015 – o momento,"Belo Horizonte e Região, Brasil",Investor and Board Member,0,"[out, de, 2015, –, o, momento]",2015-10-01,2017-09-01,False,23.0,1,23.0,0,0,NaN,NaN
8,Any Company,out de 2012 – mai de 2013,"Santiago, Chile",Start-Up Chile Entrepreneur,0,"[out, de, 2012, –, mai, de, 2013]",2012-10-01,2013-05-01,False,7.0,1,7.0,0,0,NaN,NaN
9,Any Company,set de 2009 – jan de 2011,"São Paulo e Região, Brasil",Founder and Sales Director,0,"[set, de, 2009, –, jan, de, 2011]",2009-09-01,2011-01-01,False,16.0,1,16.0,0,0,NaN,NaN
10,Trojan Technologies,jul de 2017 – o momento,"Belo Horizonte e Região, Brasil",Product Manager,1,"[jul, de, 2017, –, o, momento]",2017-07-01,2017-09-01,True,2.0,1,2.0,0,0,NaN,NaN
11,Any Company,jan de 2017 – o momento,"Belo Horizonte, Minas Gerais",Diretora de Whatever,1,"[jan, de, 2017, –, o, momento]",2017-01-01,2017-09-01,False,8.0,1,8.0,0,0,NaN,NaN
12,Trojan Technologies,mar de 2017 – jul de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de Onboarding,1,"[mar, de, 2017, –, jul, de, 2017]",2017-03-01,2017-07-01,False,4.0,1,4.0,0,0,NaN,NaN
13,Any Company,dez de 2016 – mar de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de aceleração,1,"[dez, de, 2016, –, mar, de, 2017]",2016-12-01,2017-03-01,False,3.0,1,3.0,0,0,NaN,NaN
14,Any Company,jun de 2016 – jan de 2017,"Belo Horizonte e Região, Brasil",Subcoordenadora de Desenvolvimento de Negócios...,1,"[jun, de, 2016, –, jan, de, 2017]",2016-06-01,2017-01-01,False,7.0,1,7.0,0,0,NaN,NaN


In [8]:
#Preenche as colunas de tempo, de cargo atual e de data de admissão
experiencia.loc[experiencia.company.str.contains('Any',case=False),'tempo_empresa_meses_any']=experiencia.duracao
experiencia.loc[experiencia.company.str.contains('Trojan',case=False),'tempo_empresa_meses_trojan']=experiencia.duracao
experiencia.loc[experiencia.atual == True,'cargo_atual']=experiencia.position
experiencia['inicio_str']=experiencia.inicio

def date_to_string(x):
    """Função que converte datetime para string"""
    return x.strftime('%Y/%m/%d')

experiencia['inicio_str']=experiencia['inicio_str'].map(date_to_string)
experiencia.loc[experiencia.atual == True,'data_admissao']=experiencia.inicio_str
experiencia.location.fillna('Não Informado', inplace=True)

#Mostra rsultado
experiencia.head(10)

,company,date,location,position,perfil_id,data_controle,inicio,fim,atual,duracao,qtd_empregos,tempo_mercado_meses,tempo_empresa_meses_any,tempo_empresa_meses_trojan,cargo_atual,data_admissao,inicio_str
0,Trojan Technologies,set de 2011 – o momento,"Belo Horizonte e Região, Brasil",CEO and Founder,0,"[set, de, 2011, –, o, momento]",2011-09-01,2017-09-01,True,72.0,1,72.0,0.0,72.0,CEO and Founder,2011/09/01,2011/09/01
6,Any Company,mar de 2016 – o momento,Não Informado,Endeavor Entrepreneur,0,"[mar, de, 2016, –, o, momento]",2016-03-01,2017-09-01,False,18.0,1,18.0,18.0,0.0,NaN,NaN,2016/03/01
7,Any Company,out de 2015 – o momento,"Belo Horizonte e Região, Brasil",Investor and Board Member,0,"[out, de, 2015, –, o, momento]",2015-10-01,2017-09-01,False,23.0,1,23.0,23.0,0.0,NaN,NaN,2015/10/01
8,Any Company,out de 2012 – mai de 2013,"Santiago, Chile",Start-Up Chile Entrepreneur,0,"[out, de, 2012, –, mai, de, 2013]",2012-10-01,2013-05-01,False,7.0,1,7.0,7.0,0.0,NaN,NaN,2012/10/01
9,Any Company,set de 2009 – jan de 2011,"São Paulo e Região, Brasil",Founder and Sales Director,0,"[set, de, 2009, –, jan, de, 2011]",2009-09-01,2011-01-01,False,16.0,1,16.0,16.0,0.0,NaN,NaN,2009/09/01
10,Trojan Technologies,jul de 2017 – o momento,"Belo Horizonte e Região, Brasil",Product Manager,1,"[jul, de, 2017, –, o, momento]",2017-07-01,2017-09-01,True,2.0,1,2.0,0.0,2.0,Product Manager,2017/07/01,2017/07/01
11,Any Company,jan de 2017 – o momento,"Belo Horizonte, Minas Gerais",Diretora de Whatever,1,"[jan, de, 2017, –, o, momento]",2017-01-01,2017-09-01,False,8.0,1,8.0,8.0,0.0,NaN,NaN,2017/01/01
12,Trojan Technologies,mar de 2017 – jul de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de Onboarding,1,"[mar, de, 2017, –, jul, de, 2017]",2017-03-01,2017-07-01,False,4.0,1,4.0,0.0,4.0,NaN,NaN,2017/03/01
13,Any Company,dez de 2016 – mar de 2017,"Belo Horizonte, Minas Gerais",Coordenadora de aceleração,1,"[dez, de, 2016, –, mar, de, 2017]",2016-12-01,2017-03-01,False,3.0,1,3.0,3.0,0.0,NaN,NaN,2016/12/01
14,Any Company,jun de 2016 – jan de 2017,"Belo Horizonte e Região, Brasil",Subcoordenadora de Desenvolvimento de Negócios...,1,"[jun, de, 2016, –, jan, de, 2017]",2016-06-01,2017-01-01,False,7.0,1,7.0,7.0,0.0,NaN,NaN,2016/06/01


In [9]:
#Cria uma base para quantidade de emprego e de tempo
q_t=experiencia.groupby('perfil_id').sum()
q_t=q_t.drop(['duracao','atual',],axis=1)
q_t['key']=q_t.index

#Mostra resultado
q_t.head(10)

,qtd_empregos,tempo_mercado_meses,tempo_empresa_meses_any,tempo_empresa_meses_trojan,key
perfil_id,,,,,
0,5,136.0,64.0,72.0,0
1,5,24.0,18.0,6.0,1
2,5,42.0,32.0,10.0,2
3,3,11.0,5.0,6.0,3
4,5,12.0,1.0,11.0,4
5,5,113.0,86.0,27.0,5
6,5,204.0,184.0,20.0,6
7,2,23.0,18.0,5.0,7
8,5,136.0,56.0,80.0,8


## 05.2 - Cargo atual na empresa


In [10]:
#Cria base de cargo atual
c_a=pd.DataFrame(experiencia.cargo_atual)
c_a['key']=experiencia.perfil_id
c_a.loc[c_a.cargo_atual.isnull()==True,'key']=np.nan
c_a=c_a[(c_a.cargo_atual.notnull()) & (c_a.key.notnull())]

#Mostra resultado
c_a.head(10)

,cargo_atual,key
0,CEO and Founder,0.0
10,Product Manager,1.0
16,Full Stack Developer,2.0
20,CRM Analyst,3.0
24,Sales Operations Coordinator,4.0
29,Key Account Manager,5.0
34,Software Engineer and Data Scientist,6.0
35,Data Engineer,6.0
39,Aprediz administrativo,7.0
41,Founder and CFO,8.0


In [11]:
#Prepara a resposta
resposta05=pd.merge(q_t,c_a,how='outer',on=['key'])

#Mostra resultado
resposta05.head(10)

,qtd_empregos,tempo_mercado_meses,tempo_empresa_meses_any,tempo_empresa_meses_trojan,key,cargo_atual
0,5,136.0,64.0,72.0,0,CEO and Founder
1,5,24.0,18.0,6.0,1,Product Manager
2,5,42.0,32.0,10.0,2,Full Stack Developer
3,3,11.0,5.0,6.0,3,CRM Analyst
4,5,12.0,1.0,11.0,4,Sales Operations Coordinator
5,5,113.0,86.0,27.0,5,Key Account Manager
6,5,204.0,184.0,20.0,6,Software Engineer and Data Scientist
7,5,204.0,184.0,20.0,6,Data Engineer
8,2,23.0,18.0,5.0,7,Aprediz administrativo
9,5,136.0,56.0,80.0,8,Founder and CFO


## 05.3 - Setor do Funcionário na empresa

A partir da posicao dos funcionários, é possível descobrir (ou inferir) o setor que eles trabalham.

In [12]:
#Vetores com palavras-cahves para a função contains
v_setfina=['DP','financeiro','finanças','controladoria','Financeiro','Finance']
v_setrh=['People','Time','Public','Onboarding','Talent','Pessoas']
v_setti=['Tech','Data','tecnologia','Arquiteto','Architect']
v_setdespro=['Desenvolvedor','Programador','Desenvolvedora','Developer']
v_seteng=['Engineer','Engenheiro']
v_setdesart=['arte','Designer']
v_setcom=['CRM','Sales','Key','Marketing','Customer','Branding','Business',\
          'Growth','Purchasing','CS','Commercial','marketing','Atendimento',\
          'vendas','Costomer','Comercial']
v_setadm=['administrativo','CEO','Founder','Store Operation','ERP','COO','In-Store',\
          'operações','Executivo','Executiva','Executive','Regional','Sao','Head Of Strategy','Product']

def troca_string_pt1(vetor_string, df_controle, valor):
    """"Função que retorna um vetor do valor pretendido
    utilizando a função contains"""
    r_vetor= [np.nan for x in range(len(df_controle))]
    for_vetor=np.arange(len(df_controle))
    for x in vetor_string:
        T_F=df_controle.str.contains(x)
        for y in for_vetor:
            if T_F[y]==True:
                r_vetor[y]=valor
    return r_vetor

def troca_string_pt2(vetor_string, r_vetor, df_controle, valor):
    """"Função que retorna um vetor do valor pretendido
    utilizando a função contains"""
    for_vetor=np.arange(len(df_controle))
    for x in vetor_string:
        T_F=df_controle.str.contains(x)
        for y in for_vetor:
            if T_F[y]==True:
                r_vetor[y]=valor
    return r_vetor

#Cria a coluna:departamento
p1=troca_string_pt1(v_setfina,resposta05.cargo_atual,'Financeiro')
p2=troca_string_pt2(v_setrh,p1,resposta05.cargo_atual,'Recursos Humanos')
p3=troca_string_pt2(v_setti,p2,resposta05.cargo_atual,'TI')
p4=troca_string_pt2(v_setdespro,p3,resposta05.cargo_atual,'Desenvolvedor / Programador')
p5=troca_string_pt2(v_seteng,p4,resposta05.cargo_atual,'Engenharia')
p6=troca_string_pt2(v_setdesart,p5,resposta05.cargo_atual,'Design / Arte')
p7=troca_string_pt2(v_setcom,p6,resposta05.cargo_atual,'Comercial')
resposta05['departamento']=troca_string_pt2(v_setadm,p7,resposta05.cargo_atual,'Administrativo')

#Mostra resultado
resposta05.head(10)

,qtd_empregos,tempo_mercado_meses,tempo_empresa_meses_any,tempo_empresa_meses_trojan,key,cargo_atual,departamento
0,5,136.0,64.0,72.0,0,CEO and Founder,Administrativo
1,5,24.0,18.0,6.0,1,Product Manager,Administrativo
2,5,42.0,32.0,10.0,2,Full Stack Developer,Desenvolvedor / Programador
3,3,11.0,5.0,6.0,3,CRM Analyst,Comercial
4,5,12.0,1.0,11.0,4,Sales Operations Coordinator,Comercial
5,5,113.0,86.0,27.0,5,Key Account Manager,Comercial
6,5,204.0,184.0,20.0,6,Software Engineer and Data Scientist,Engenharia
7,5,204.0,184.0,20.0,6,Data Engineer,Engenharia
8,2,23.0,18.0,5.0,7,Aprediz administrativo,Administrativo
9,5,136.0,56.0,80.0,8,Founder and CFO,Administrativo


In [13]:
#Cria base data de admissão
d_a=pd.DataFrame(experiencia.data_admissao)
d_a['key']=experiencia.perfil_id
d_a.loc[d_a.data_admissao.isnull()==True,'key']=np.nan
d_a=d_a[(d_a.data_admissao.notnull()) & (d_a.key.notnull())]

#Mostra resulatdo
d_a.head(10)

,data_admissao,key
0,2011/09/01,0.0
10,2017/07/01,1.0
16,2016/11/01,2.0
20,2017/08/01,3.0
24,2017/03/01,4.0
29,2016/07/01,5.0
34,2016/11/01,6.0
35,2016/11/01,6.0
39,2017/04/01,7.0
41,2011/01/01,8.0


In [14]:
def string_to_date(x):
    """Função que converte datetime para string"""
    return datetime.strptime(x,'%Y/%m/%d')
d_a['data_admissao']=d_a['data_admissao'].map(string_to_date)

#Mostra resultado
d_a.head(10)

,data_admissao,key
0,2011-09-01,0.0
10,2017-07-01,1.0
16,2016-11-01,2.0
20,2017-08-01,3.0
24,2017-03-01,4.0
29,2016-07-01,5.0
34,2016-11-01,6.0
35,2016-11-01,6.0
39,2017-04-01,7.0
41,2011-01-01,8.0


In [15]:
#Prepara a resposta
resposta05=pd.merge(resposta05,d_a,how='outer',on=['key'])
resposta05['perfil_id']=resposta05.key
resposta05=resposta05.drop(['key'],axis=1)

#Mostra a resposta
resposta05.head(10)

,qtd_empregos,tempo_mercado_meses,tempo_empresa_meses_any,tempo_empresa_meses_trojan,cargo_atual,departamento,data_admissao,perfil_id
0,5,136.0,64.0,72.0,CEO and Founder,Administrativo,2011-09-01,0
1,5,24.0,18.0,6.0,Product Manager,Administrativo,2017-07-01,1
2,5,42.0,32.0,10.0,Full Stack Developer,Desenvolvedor / Programador,2016-11-01,2
3,3,11.0,5.0,6.0,CRM Analyst,Comercial,2017-08-01,3
4,5,12.0,1.0,11.0,Sales Operations Coordinator,Comercial,2017-03-01,4
5,5,113.0,86.0,27.0,Key Account Manager,Comercial,2016-07-01,5
6,5,204.0,184.0,20.0,Software Engineer and Data Scientist,Engenharia,2016-11-01,6
7,5,204.0,184.0,20.0,Software Engineer and Data Scientist,Engenharia,2016-11-01,6
8,5,204.0,184.0,20.0,Data Engineer,Engenharia,2016-11-01,6
9,5,204.0,184.0,20.0,Data Engineer,Engenharia,2016-11-01,6


In [16]:
#Remove duplicatas das respostas
resposta05=resposta05.drop([7,8,9,15,16,17,19,20,21,27,28,29,45,46,47,54,55,56,60,61,62,87,88,89])

#Mostra resultado
resposta05.head(10)

,qtd_empregos,tempo_mercado_meses,tempo_empresa_meses_any,tempo_empresa_meses_trojan,cargo_atual,departamento,data_admissao,perfil_id
0,5,136.0,64.0,72.0,CEO and Founder,Administrativo,2011-09-01,0
1,5,24.0,18.0,6.0,Product Manager,Administrativo,2017-07-01,1
2,5,42.0,32.0,10.0,Full Stack Developer,Desenvolvedor / Programador,2016-11-01,2
3,3,11.0,5.0,6.0,CRM Analyst,Comercial,2017-08-01,3
4,5,12.0,1.0,11.0,Sales Operations Coordinator,Comercial,2017-03-01,4
5,5,113.0,86.0,27.0,Key Account Manager,Comercial,2016-07-01,5
6,5,204.0,184.0,20.0,Software Engineer and Data Scientist,Engenharia,2016-11-01,6
10,2,23.0,18.0,5.0,Aprediz administrativo,Administrativo,2017-04-01,7
11,5,136.0,56.0,80.0,Founder and CFO,Administrativo,2011-01-01,8
12,3,55.0,20.0,35.0,Sales Operations Manager,Comercial,2016-08-01,9


## 05.4 Salvando Base Tratada

A base tratada será gravada no arquivo **experiencia_trabalho-tratado.csv**.

As informações consolidadas por perfil serão gravadas na base **perfil-experiencia,csv** contendo as colunas: **perfil_id**, **posicao_atual**, **data_admissao**, **departamento**, **tempo_empresa_meses**, **tempo_mercado_meses**, **qtd_empregos**.

In [17]:
# Grava a experiencia tratada em arquivo
experiencia_tratado=experiencia[['perfil_id','company','location','position','inicio','fim','duracao','atual']].copy().set_index('perfil_id')
experiencia_tratado.to_csv('bases/experiencia-tratado.csv')

# Grava o consolidade da experiencia por perfil em arquivo
experiencia_perfil=resposta05[['perfil_id','cargo_atual','data_admissao','departamento','tempo_empresa_meses_trojan','tempo_mercado_meses','qtd_empregos']].copy().set_index('perfil_id')
experiencia_perfil.to_csv('bases/perfil-experiencia.csv')

### Navegação:
* [Voltar para a análise](00-analise-dos-dados.ipynb)